# Basic Example Notebook
Hello and welcome to `PauliPropagation.jl` - a Julia package for Pauli propagation simulation of quantum circuits and quantum systems.

That is, we estimate quantities like
$ Tr[\rho \ \mathcal{E}(\hat{O})] $
where $\rho$ is an initial state, $\mathcal{E}$ is a quantum channel that is not necessarily unitary, and $\hat{O}$ is an observable preferrably sparse in Pauli basis.

In the following we are going to introduce the basic concepts and datatypes of the package.

In [1]:
using PauliPropagation

Define the number of qubits in the simulation.

In [2]:
nq = 64

64

Now define the observable $\hat{O} = Z_{32} = I_1 \otimes ... \otimes I_{31} \otimes Z_{32} \otimes I_{33}... \otimes I_{62}$.

The `PauliString` type is our high-level way of handling Pauli strings. This above constructor defines a `PauliString` that has the only non-Identity Pauli at position 32.

This library uses encodes Paulis into the bits of Integers for performance reasons, and also actions of gates onto Paulis are defined as bit operations. You can definitely get by just sticking to the high level interface. Check out some other example notebooks for more involved code.

In [3]:
pstr = PauliString(nq, :Z, 32)

PauliString(nqubits: 64, 1.0 * IIIIIIIIIIIIIIIIIIII...)

Now we specify the circuit that we want to run. We provide some simple constructors for common circuit ansätze. Here we use a generic circuit ansätz called the `hardwareefficientcircuit`. It consists of repeated RX- RZ - RX Pauli rotation gates intertwined with RYY entangling gates.

For everything to work out, we need to specify the number of circuit layers and the entangling topology. By defailt we use the so-called 1D `bricklayertopology`, but you can customize a topology via an array or tuples like `[(1, 2), (1, 3), ...]`.

In [4]:
nl = 4
topology = bricklayertopology(nq; periodic=false)
circuit = hardwareefficientcircuit(nq, nl; topology=topology)
nparams = countparameters(circuit)

1020

This circuit has a total of 1020 Pauli rotation gates. Let us define some random vector of parameters.

In [5]:
using Random
Random.seed!(42)
thetas = randn(nparams);

Now we get to the interesting part. The propagation of the Observable `pstr` through `circuit`. In other words, we apply the circuit onto the Pauli string.

First, let us do it exactly.

In [6]:
@time psum = propagate(circuit, pstr, thetas)

  0.686875 seconds (594.09 k allocations: 83.708 MiB, 1.35% gc time, 43.52% compilation time)


PauliSum(nqubits: 64, 164692 Pauli terms:
 4.027e-9 * IIIIIIIIIIIIIIIIIIII...
 1.0319e-7 * IIIIIIIIIIIIIIIIIIII...
 -5.3954e-9 * IIIIIIIIIIIIIIIIIIII...
 5.4961e-5 * IIIIIIIIIIIIIIIIIIII...
 -1.9635e-10 * IIIIIIIIIIIIIIIIIIII...
 6.1435e-5 * IIIIIIIIIIIIIIIIIIII...
 3.0334e-8 * IIIIIIIIIIIIIIIIIIII...
 -2.7134e-9 * IIIIIIIIIIIIIIIIIIII...
 5.3742e-9 * IIIIIIIIIIIIIIIIIIII...
 -2.0534e-7 * IIIIIIIIIIIIIIIIIIII...
 -1.6002e-6 * IIIIIIIIIIIIIIIIIIII...
 -4.3295e-8 * IIIIIIIIIIIIIIIIIIII...
 1.6012e-8 * IIIIIIIIIIIIIIIIIIII...
 4.5844e-10 * IIIIIIIIIIIIIIIIIIII...
 -2.4033e-8 * IIIIIIIIIIIIIIIIIIII...
 7.8656e-7 * IIIIIIIIIIIIIIIIIIII...
 -9.4308e-5 * IIIIIIIIIIIIIIIIIIII...
 -2.3263e-6 * IIIIIIIIIIIIIIIIIIII...
 -1.4941e-8 * IIIIIIIIIIIIIIIIIIII...
 2.2601e-9 * IIIIIIIIIIIIIIIIIIII...
  ⋮)

Just to run it again after Julia's jit-compilation:

In [7]:
@time propagate(circuit, pstr, thetas);

  0.387542 seconds (2.21 k allocations: 45.745 MiB, 1.13% gc time)


How is it possible that we can exactly run 64-qubit circuits? We leverage a concept referred to as the _entanglement lightcone_. But we don't do it manually, the Pauli propagation framework does it naturally. Still, we created over 160k Pauli strings in an object of type `PauliSum`. As the name suggests, this is a sum of Pauli strings. Be very mindful of the fact that the runtime will initially scale exponentially with the circuit depth!

The `psum` object may already be of interest of you. But here is a quick and easy way how to evaluate the expectation value if initial state is the zero-state, i.e. $\rho |0\langle\rangle 0|$.

In [8]:
overlapwithzero(psum)

0.3196162959450149

Remember, this was an exact calculation. Well actually, our library has a default truncation threshold of Pauli strings with coefficients smaller than 1e-10. Here is how you control the minimum absolute coefficient threshold `min_abs_coeff`.

In [9]:
@time psum_exact = propagate(circuit, pstr, thetas; min_abs_coeff = 0)

  0.459919 seconds (80.22 k allocations: 50.950 MiB, 1.35% gc time, 16.26% compilation time)


PauliSum(nqubits: 64, 173055 Pauli terms:
 4.0274e-9 * IIIIIIIIIIIIIIIIIIII...
 1.0319e-7 * IIIIIIIIIIIIIIIIIIII...
 -5.4141e-9 * IIIIIIIIIIIIIIIIIIII...
 5.4961e-5 * IIIIIIIIIIIIIIIIIIII...
 -1.9553e-10 * IIIIIIIIIIIIIIIIIIII...
 6.1435e-5 * IIIIIIIIIIIIIIIIIIII...
 3.0334e-8 * IIIIIIIIIIIIIIIIIIII...
 -2.6915e-9 * IIIIIIIIIIIIIIIIIIII...
 5.3999e-9 * IIIIIIIIIIIIIIIIIIII...
 -2.0534e-7 * IIIIIIIIIIIIIIIIIIII...
 -1.6002e-6 * IIIIIIIIIIIIIIIIIIII...
 -4.3295e-8 * IIIIIIIIIIIIIIIIIIII...
 1.6008e-8 * IIIIIIIIIIIIIIIIIIII...
 4.5844e-10 * IIIIIIIIIIIIIIIIIIII...
 -2.4032e-8 * IIIIIIIIIIIIIIIIIIII...
 7.8656e-7 * IIIIIIIIIIIIIIIIIIII...
 -9.4308e-5 * IIIIIIIIIIIIIIIIIIII...
 -2.3263e-6 * IIIIIIIIIIIIIIIIIIII...
 -1.4951e-8 * IIIIIIIIIIIIIIIIIIII...
 2.2597e-9 * IIIIIIIIIIIIIIIIIIII...
  ⋮)

And the expectation value is pretty much the same.

In [10]:
print("The error with our default truncation of `1e-10` is ", abs(overlapwithzero(psum_exact) - overlapwithzero(psum)))

The error with our default truncation of `1e-10` is 3.773479306801164e-10

How about less strict truncation thresholds?

In [11]:
min_abs_coeff = 1e-3
@time psum_coeff = propagate(circuit, pstr, thetas; min_abs_coeff = min_abs_coeff)

  0.043798 seconds (61.19 k allocations: 5.179 MiB, 86.97% compilation time)


PauliSum(nqubits: 64, 1704 Pauli terms:
 0.0010765 * IIIIIIIIIIIIIIIIIIII...
 0.0017082 * IIIIIIIIIIIIIIIIIIII...
 0.01189 * IIIIIIIIIIIIIIIIIIII...
 -0.0029782 * IIIIIIIIIIIIIIIIIIII...
 -0.001331 * IIIIIIIIIIIIIIIIIIII...
 0.0045197 * IIIIIIIIIIIIIIIIIIII...
 0.0013986 * IIIIIIIIIIIIIIIIIIII...
 0.0023729 * IIIIIIIIIIIIIIIIIIII...
 -0.0053276 * IIIIIIIIIIIIIIIIIIII...
 0.052407 * IIIIIIIIIIIIIIIIIIII...
 -0.0045722 * IIIIIIIIIIIIIIIIIIII...
 -0.0095782 * IIIIIIIIIIIIIIIIIIII...
 -0.0092645 * IIIIIIIIIIIIIIIIIIII...
 -0.012861 * IIIIIIIIIIIIIIIIIIII...
 -0.0052605 * IIIIIIIIIIIIIIIIIIII...
 0.0058996 * IIIIIIIIIIIIIIIIIIII...
 0.0019707 * IIIIIIIIIIIIIIIIIIII...
 -0.0099958 * IIIIIIIIIIIIIIIIIIII...
 0.010155 * IIIIIIIIIIIIIIIIIIII...
 -0.014565 * IIIIIIIIIIIIIIIIIIII...
  ⋮)

In [12]:
print("The error with `min_abs_coeff = $min_abs_coeff` is ", abs(overlapwithzero(psum_exact) - overlapwithzero(psum_coeff)))

The error with `min_abs_coeff = 0.001` is 0.004851230411391183

The expectation value is still surprisingly precise.

Another very general but powerful truncation is one based on _Pauli weight_. We can truncate Pauli strings that have many non-Identity Paulis, which has been proven to be a valid truncation for random circuits, but it also works well in practice. We can pass this as the keyword argument `max_weight`.

In [13]:
max_weight = 5
@time psum_weight = propagate(circuit, pstr, thetas; max_weight = max_weight)

  0.108168 seconds (81.40 k allocations: 8.104 MiB, 46.64% compilation time)


PauliSum(nqubits: 64, 16453 Pauli terms:
 -1.6207e-5 * IIIIIIIIIIIIIIIIIIII...
 0.011987 * IIIIIIIIIIIIIIIIIIII...
 7.3032e-5 * IIIIIIIIIIIIIIIIIIII...
 7.8792e-8 * IIIIIIIIIIIIIIIIIIII...
 6.5435e-10 * IIIIIIIIIIIIIIIIIIII...
 -9.1272e-9 * IIIIIIIIIIIIIIIIIIII...
 1.8143e-8 * IIIIIIIIIIIIIIIIIIII...
 1.7645e-10 * IIIIIIIIIIIIIIIIIIII...
 0.00019728 * IIIIIIIIIIIIIIIIIIII...
 0.00018569 * IIIIIIIIIIIIIIIIIIII...
 0.00034955 * IIIIIIIIIIIIIIIIIIII...
 7.9429e-6 * IIIIIIIIIIIIIIIIIIII...
 0.0016765 * IIIIIIIIIIIIIIIIIIII...
 -1.2376e-8 * IIIIIIIIIIIIIIIIIIII...
 2.6325e-7 * IIIIIIIIIIIIIIIIIIII...
 -0.00047972 * IIIIIIIIIIIIIIIIIIII...
 6.0018e-10 * IIIIIIIIIIIIIIIIIIII...
 0.00088951 * IIIIIIIIIIIIIIIIIIII...
 -4.7825e-6 * IIIIIIIIIIIIIIIIIIII...
 -9.0788e-5 * IIIIIIIIIIIIIIIIIIII...
  ⋮)

In [14]:
print("The error with `max_weight = $max_weight` is ", abs(overlapwithzero(psum_exact) - overlapwithzero(psum_weight)))

The error with `max_weight = 5` is 0.0018570230051821457

One mindset to adopt when using this package (or any other computational physics package for that matter), is that exact computation will quickly be infeasible. Truncations introduce a necessary trade-off between computational cost and accuracy.